## Dataset snapshot

### Train / Test Presplit

In [1]:
import os, sys
sys.path.append(os.path.join(os.path.abspath(''), '../../'))
sys.path.append(os.path.join(os.path.abspath(''), '../../toyDb'))

import numpy as np
import json
from toyDb.databases import ExperimentDb
ExperimentDb.init_from_default_db()

def getShaderIdSplit() -> 'np.array':
  # Good, let's hard code the submission ddl for I3D'24
  RNGSeed = 20240105
  TRAIN_RATIO = 0.8

  CACHE = os.path.join(os.path.abspath(''), 'shaderIdSplitCache.json')

  # this is complete (than ImageOnlyShader)
  allShaderIdQuery = ExperimentDb.ImageOnlyExperiment.select(
    ExperimentDb.ImageOnlyExperiment.shader_shadertoy_id
  ).order_by(ExperimentDb.ImageOnlyExperiment.shader_shadertoy_id.asc()).distinct()

  # the db in use have 20669 is_imageonly shaders
  print(f"Shaders that took part in the partition: {len(allShaderIdQuery)}")
  assert(len(allShaderIdQuery) == 20669)

  rng = np.random.default_rng(RNGSeed)

  shaderIds = np.array([i.shader_shadertoy_id for i in allShaderIdQuery])
  shuffledIds = rng.permutation(shaderIds)
  print(shaderIds)
  print(shuffledIds)

  if not os.path.isfile(CACHE):  
    with open(CACHE, "w") as fp:
      json.dump(shuffledIds.tolist(), fp)
  else:
    with open(CACHE, "r") as fp:
      idFromFile = json.load(fp)
      assert(idFromFile == shuffledIds.tolist())
  
  numTrain = int(len(shuffledIds) * TRAIN_RATIO)
  numTest = len(shuffledIds) - numTrain
  print(f"numTrain: {numTrain}; numTest: {numTest}")

  return (shuffledIds[:numTrain], shuffledIds[numTrain:])

trainIds, testIds = getShaderIdSplit()

Shaders that took part in the partition: 20669
['3d23DR' '3d23Dc' '3d23R3' ... 'wtyfzz' 'wtyyWD' 'wtyyz3']
['wd3GDH' '3sXyDn' 'MdK3Dd' ... 'WltBzM' 'MsVXzc' 'sdVGWW']
numTrain: 16535; numTest: 4134


### Dataset splits for each architecture


In [3]:
from misc.ComplexDatasetSnapshotter import (
    EnvironmentFilter,
    CycleTrialsFilter,
    ErrorFilter,
    ShadertoyIdFilter,
    WidthHeightFilter,
    ResourceFilter,
    TraceAvailabilityFilter,
    SpvTokenizedLengthFilter,
    TraceDuplicationPostFilter,
    AugmentationFilter,
    ImageHashFilter,
    ComplexDatasetSnapshotter
)
from misc.Directory import (
  getIntermediateDir
)

import logging
import hashlib
import pickle

logging.basicConfig(
  level=logging.INFO,
  format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

snapshotter = ComplexDatasetSnapshotter()

# Basic filters
snapshotter.registerFilter(EnvironmentFilter())
snapshotter.registerFilter(WidthHeightFilter())
snapshotter.registerFilter(ResourceFilter())
snapshotter.registerFilter(CycleTrialsFilter())
snapshotter.registerFilter(ErrorFilter())
snapshotter.registerFilter(TraceAvailabilityFilter())
snapshotter.registerFilter(ImageHashFilter())
snapshotter.registerFilter(AugmentationFilter())

lengthFilter = SpvTokenizedLengthFilter()
lengthFilter.setThreshold(4096)

if os.path.isfile(os.path.join(getIntermediateDir(), "./lengthFilterCache.json")):
    lengthFilter.readFromCache(os.path.join(getIntermediateDir(), "./lengthFilterCache.json"))
else:
    lengthFilter.process(parallel=True)
    lengthFilter.writeToCache(os.path.join(getIntermediateDir(), "./lengthFilterCache.json"))

snapshotter.registerFilter(lengthFilter)

# Train / test split filter
shdrIdFilter = ShadertoyIdFilter()
shdrIdFilter.registerGroup("trainShdrExprs", trainIds.tolist())
shdrIdFilter.registerGroup("testShdrExprs", testIds.tolist())
snapshotter.registerFilter(shdrIdFilter)

snapshotter.examineGroups(1)

2023-12-22 16:03:39,110 - misc.ComplexDatasetSnapshotter - INFO - White hash for (1024, 768): 908b6cfc9aef496dd5ab5c5540d80c6383ed6e92f86044574c996315381bc064
2023-12-22 16:03:39,111 - misc.ComplexDatasetSnapshotter - INFO - Transparent black hash for (1024, 768): bbd05cf6097ac9b1f89ea29d2542c1b7b67ee46848393895f5a9e43fa1f621e5
2023-12-22 16:03:39,118 - misc.ComplexDatasetSnapshotter - INFO - White hash for (800, 600): d883267b40e389a772f00ef4c50d49471138afed85da8f101de16ad6cf5a9d9f
2023-12-22 16:03:39,119 - misc.ComplexDatasetSnapshotter - INFO - Transparent black hash for (800, 600): 124617c1f65e92d3bc895fbd869e4bb16a30754b198f59e6e973949b9aaa1b01
2023-12-22 16:03:39,148 - misc.ComplexDatasetSnapshotter - INFO - White hash for (1920, 1080): 598ddbfa658eaf70a2e0c50fa12f914c28a4496bbb150a21d6d77d73b0d8c55d
2023-12-22 16:03:39,149 - misc.ComplexDatasetSnapshotter - INFO - Transparent black hash for (1920, 1080): 788ae0147bdf979a6575938ca2d7d4403788588f7be2010f03776c968fd1ab49


  0%|          | 0/28278 [00:00<?, ?it/s]

  0%|          | 0/28278 [00:00<?, ?it/s]

EnvironmentFilter_EnvId1 = 447888
EnvironmentFilter_EnvId2 = 20669
EnvironmentFilter_EnvId3 = 20669
EnvironmentFilter_EnvId4 = 20669
EnvironmentFilter_EnvId5 = 20669
EnvironmentFilter_EnvId6 = 20669
WidthHeightFilter_1024-768 = 509895
WidthHeightFilter_800-600 = 20669
WidthHeightFilter_1920-1080 = 20669
ResourceFilter_resource1 = 97784
ResourceFilter_resourceNone = 175979
ResourceFilter_resource2 = 13878
ResourceFilter_resource3 = 13873
ResourceFilter_resource4 = 13873
ResourceFilter_resource5 = 13875
ResourceFilter_resource6 = 13876
ResourceFilter_resource7 = 13872
ResourceFilter_resource8 = 13867
ResourceFilter_resource9 = 13872
ResourceFilter_resource10 = 13881
ResourceFilter_resource11 = 13873
ResourceFilter_resource12 = 13879
ResourceFilter_resource13 = 13871
ResourceFilter_resource14 = 13869
ResourceFilter_resource15 = 13868
ResourceFilter_resource16 = 13874
ResourceFilter_resource17 = 13874
ResourceFilter_resource18 = 13872
ResourceFilter_resource19 = 13869
ResourceFilter_resour

In [4]:
canonicalFilter = [
    [('CycleTrialsFilter', '30cycles-10trials')],
    [('WidthHeightFilter', '1024-768')],
    [('AugmentationFilter', 'aug0')]
]

commonAdditionalFilters = [
    [('ResourceFilter', 'resource1')],
    [('ErrorFilter', 'error0')],
    [('TraceAvailabilityFilter', 'haveTrace')],
    [('ImageHashFilter', 'normalHash')],
    [('SpvTokenizedLengthFilter', 'belowOrEqualThreshold4096')]
]

candidateDatasets = {
    "RTX3060": {
        "baseFilters": canonicalFilter + [[('EnvironmentFilter', 'EnvId1')]],
        "additionalFilters": commonAdditionalFilters,
        "trainAdditionalFilters": [
            [('ShadertoyIdFilter', 'trainShdrExprs')]
        ],
        "testAdditionalFilters": [
            [('ShadertoyIdFilter', 'testShdrExprs')]
        ]
    },
    "RX6600XT": {
        "baseFilters": canonicalFilter + [[('EnvironmentFilter', 'EnvId2')]],
        "additionalFilters": commonAdditionalFilters,
        "trainAdditionalFilters": [
            [('ShadertoyIdFilter', 'trainShdrExprs')]
        ],
        "testAdditionalFilters": [
            [('ShadertoyIdFilter', 'testShdrExprs')]
        ]
    },
    "UHD630": {
        "baseFilters": canonicalFilter + [[('EnvironmentFilter', 'EnvId3')]],
        "additionalFilters": commonAdditionalFilters,
        "trainAdditionalFilters": [
            [('ShadertoyIdFilter', 'trainShdrExprs')]
        ],
        "testAdditionalFilters": [
            [('ShadertoyIdFilter', 'testShdrExprs')]
        ]
    },
    "RTX4060Laptop": {
        "baseFilters": canonicalFilter + [[('EnvironmentFilter', 'EnvId4')]],
        "additionalFilters": commonAdditionalFilters,
        "trainAdditionalFilters": [
            [('ShadertoyIdFilter', 'trainShdrExprs')]
        ],
        "testAdditionalFilters": [
            [('ShadertoyIdFilter', 'testShdrExprs')]
        ]
    },
    "RenoirAPU": {
        "baseFilters": canonicalFilter + [[('EnvironmentFilter', 'EnvId5')]],
        # Will print verbose percentages regarding the additional filters
        "additionalFilters": commonAdditionalFilters,
        "trainAdditionalFilters": [
            [('ShadertoyIdFilter', 'trainShdrExprs')]
        ],
        "testAdditionalFilters": [
            [('ShadertoyIdFilter', 'testShdrExprs')]
        ]
    },
    "GTX1660": {
        "baseFilters": canonicalFilter + [[('EnvironmentFilter', 'EnvId6')]],
        # Will print verbose percentages regarding the additional filters
        "additionalFilters": [
            [('ResourceFilter', 'resource1')],
            [('ErrorFilter', 'error5')],
            [('TraceAvailabilityFilter', 'haveTrace')],
            [('ImageHashFilter', 'normalHash')],
            [('SpvTokenizedLengthFilter', 'belowOrEqualThreshold4096')]
        ],
        "trainAdditionalFilters": [
            [('ShadertoyIdFilter', 'trainShdrExprs')]
        ],
        "testAdditionalFilters": [
            [('ShadertoyIdFilter', 'testShdrExprs')]
        ]
    },
}

for setName, setDesc in candidateDatasets.items():
    outputName = f"FragPerfSnapshotTracedFinalDataset-{setName}.dat"
    outputPath = os.path.join(getIntermediateDir(), f"./{outputName}")
    print(f"Dataset {outputName}:")

    # first we go interpretation
    baseResults = snapshotter.evalFilters(setDesc['baseFilters'])
    
    print(f"=> Train Set Filter Interpretation:")
    trainExprIds = snapshotter.interpretFilters(
        set(baseResults), setDesc['additionalFilters'] + setDesc['trainAdditionalFilters']
    )

    print(f"=> Test Set Filter Interpretation:")
    testExprIds = snapshotter.interpretFilters(
        set(baseResults), setDesc['additionalFilters'] + setDesc['testAdditionalFilters']
    )

    if os.path.isfile(outputPath):
        print(f"=> file {outputPath} is already there, verify")

        with open(outputPath, "rb") as f:
            dataDict = pickle.load(f)
            if len(dataDict['train']) != len(trainExprIds):
                raise Exception(f"{outputPath} got train length {len(dataDict['train'])}, but from filters we got {len(trainExprIds)}")
            
            if len(dataDict['test']) != len(testExprIds):
                raise Exception(f"{outputPath} got test length {len(dataDict['test'])}, but from filters we got {len(testExprIds)}")
        
        del dataDict

    else:
        trainLen, testLen = snapshotter.doSnapshot(
            outputPath,
            setDesc['baseFilters'] + setDesc['additionalFilters'] + setDesc['trainAdditionalFilters'],
            setDesc['baseFilters'] + setDesc['additionalFilters'] + setDesc['testAdditionalFilters']
        )
        assert(len(trainExprIds) == trainLen)
        assert(len(testExprIds) == testLen)

        with open(outputPath, "rb") as f:
            file_hash = hashlib.md5()
            chunk = f.read(8192)
            while chunk:
                file_hash.update(chunk)
                chunk = f.read(8192)

        print(f"Hash for {outputPath}:\n- md5sum: {file_hash.hexdigest()}")
        with open(outputPath + ".md5sum", "w") as f:
            f.write(file_hash.hexdigest())

Dataset FragPerfSnapshotTracedFinalDataset-RTX3060.dat:
=> Train Set Filter Interpretation:
  Base expr id: 392711 items
  Intersect with ResourceFilter_resource1: 13871 items
  After ResourceFilter: 13871 items
  Intersect with ErrorFilter_error0: 13871 items
  After ErrorFilter: 13871 items
  Intersect with TraceAvailabilityFilter_haveTrace: 13870 items
  After TraceAvailabilityFilter: 13870 items
  Intersect with ImageHashFilter_normalHash: 13376 items
  After ImageHashFilter: 13376 items
  Intersect with SpvTokenizedLengthFilter_belowOrEqualThreshold4096: 10794 items
  After SpvTokenizedLengthFilter: 10794 items
  Intersect with ShadertoyIdFilter_trainShdrExprs: 8606 items
  After ShadertoyIdFilter: 8606 items
  Returning expr id: 8606 items
=> Test Set Filter Interpretation:
  Base expr id: 392711 items
  Intersect with ResourceFilter_resource1: 13871 items
  After ResourceFilter: 13871 items
  Intersect with ErrorFilter_error0: 13871 items
  After ErrorFilter: 13871 items
  Inter